In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

import pywt
from scipy.special import gamma
from scipy import linalg as la
from scipy import optimize as opt
from scipy import stats

import statsmodels.api as sm

###################################################################### PARAMETER/DATA LOADING  ########################################################

"""Random Seed"""
seed = 42


"""Data Loading"""
X = np.loadtxt("fBMPath2.txt")
BH05 = np.loadtxt("BHpath.05.txt")
B05 = np.loadtxt("Bpath.05.txt")
W05 = np.loadtxt("Wpath.05.txt")
BH10 = np.loadtxt("BHpath.10.txt")
B10 = np.loadtxt("Bpath.10.txt")
W10 = np.loadtxt("Wpath.10.txt")
realized_var3 = np.loadtxt('SPX3yrDailyRealizedVariance1minBins.txt')


"""Simulating Standard fBM Paths """
scaling_factor = 1000
fbm_n = 1024
fbm_H_values = [0.05, 0.5, 0.9]
T = 1


"""Simulating RSFV """
H = 0.1
S0 = 1
V0_1 = 0.1   
nu = 1
rho = -0.65    


"""MC MLE StdBM Bias"""
m_stdBM = 512
n_sim_stdBM = 100
n_steps_stdBM = 65536
H_stdBM = 0.5
p = 95
m = [16, 32, 64]


"""RSFV ATM Vol Smile"""
V0_2 = 0.04
smile_N = 252        
smile_M = 100000


"""Estimator, Models, Colors"""
models = ['Clean fBM', 'Contaminated fBM', 'RFSV', 'fOU']
estimators = ['MLE', 'Power Variation', 'Wavelet', 'Log-Log Regression']
colors = ['blue', 'green', 'red', 'purple']


"""Models C-fBM, RSFV, fOU Parameters"""
noise_level=0.2
jump_intensity=0.05 
jump_size=0.25

nu_3 = 1.5

theta=2.5
mu=0
sigma=1.0


"""MC Bias + SPX"""
H_vals = (0.1, 0.3, 0.5, 0.7, 0.9)
hist_Vals=(0.1, 0.9)
T_vals = [5]

monte_N = 256
monte_M = 50


"""Estimator Convergence"""
conv_H = 0.1
conv_T = 1

conv_N = [64, 128, 256, 512, 1024, 2048]
conv_M = 50


"""Windows"""
monte_window = 32
spx_window = 32
conv_window = 32


"""Faster Sims"""
#smile_N = 128 
#smile_M = 10000

#monte_N = 128
#monte_M = 25

#conv_N = [64, 128, 256, 512] 
#conv_M = 25

###################################################################### Main Functions #################################################################

######################################################################  Universal Functions  #

def compute_covariance_matrix(H, t):
    t1, t2 = np.meshgrid(t, t)
    
    cov = 0.5 * (t1**(2*H) + t2**(2*H) - np.abs(t1 - t2)**(2*H))
    
    return cov

def generate_fbm(H, n, T): 
    t = np.arange(1, n+1)/n
    
    cov = compute_covariance_matrix(H, t)
    L = la.cholesky(cov, lower=True)
    Z = np.random.normal(0, 1, size=n)
    clean_path = np.zeros(n+1)
    clean_path[1:] = L @ Z

    timeline = np.linspace(0, T, n+1)
    clean_scale = clean_path* (T**H)
    
    return timeline, clean_scale, clean_path
    
def confidence_interval(true_value, n, arr, p=p):
    std = np.std(arr, ddof=1)
    mean = np.mean(arr)
    
    se = std / np.sqrt(n)
    z = stats.norm.ppf(1 - (100 - p) / 200.0)
    
    ci_low = mean - z * se
    ci_high = mean + z * se
    
    bias = mean - true_value

    return mean, ci_low, ci_high, bias, std
    
######################################################################  Other Path Generators  #

def generate_c_fbm(H, n, T):
    _,_,clean_path = generate_fbm(H, n, T)

    vol = np.std(np.diff(clean_path))
    microstructure_noise = noise_level * vol * np.random.normal(0, 1, size=n+1)
    microstructure_noise[0] = 0

    total_jumps = np.random.poisson(jump_intensity * n) 
    jump_times = np.random.choice(np.arange(1, n+1), size=total_jumps, replace=False)
    jump_sizes = jump_size * vol * np.exp(np.random.normal(0, 1, size=total_jumps))
    signed_jumps = jump_sizes * np.sign(np.random.normal(0, 1, size=total_jumps))

    contaminated_path = clean_path + microstructure_noise
    contaminated_path[jump_times] += signed_jumps

    contaminated_scale = contaminated_path * (T**H)
    
    return contaminated_scale

def generate_rfsv(H, n, T, S0=S0, V0=V0_2, nu=nu_3, rho=rho):
    t, BH, _ = generate_fbm(H, n, T)
    dt = T/n
    
    B = np.zeros(n+1)
    W = np.zeros(n+1)
    dB = np.random.normal(0, np.sqrt(dt), n)
    dW = np.random.normal(0, np.sqrt(dt), n)
    B[1:] = np.cumsum(dB)
    W[1:] = np.cumsum(dW)
    
    rho_bar = np.sqrt(1 - rho**2)
    S, V = simulate_rfsv_path(BH, B, W, S0, V0, nu, rho)
    
    logV = np.log(V) 
    logV = logV - np.mean(logV) 
    
    return logV

def generate_fou(H, n, T, theta=theta, mu=mu, sigma=sigma):
    t, fbm, _ = generate_fbm(H, n, T)
    dt = T/n

    dfbm = np.diff(fbm)
    X = np.zeros(n+1)
    X[0] = mu
    
    for i in range(n):
        X[i+1] = X[i] + theta*(mu - X[i])*dt + sigma*dfbm[i]

    return X
    
######################################################################  RSFV Functions  #

def simulate_rfsv_path(BH, B, W, S0=S0, V0=V0_1, nu=nu, rho=rho):
    rho_bar = np.sqrt(1 - rho**2)
    n = len(B) - 1
    V = V0 * np.exp(nu * BH) 
    S = np.zeros(n + 1)
    
    S[0] = S0
    
    dB = np.diff(B)
    dW = np.diff(W)
    
    for i in range(n):
        sqrtV = np.sqrt(V[i])
        S[i+1] = S[i] * (1 + sqrtV * (rho * dB[i] + rho_bar * dW[i]))
    
    return S, V

def simulate_paths(n_sim=n_sim_stdBM, n_steps=n_steps_stdBM, block_size=m_stdBM, S0=S0, V0=V0_1, nu=nu, rho=rho):
    rho_bar = np.sqrt(1 - rho**2)
    dt = 1.0 / n_steps  
    estimates = []
    
    for i in range(n_sim):
        dB = np.sqrt(dt) * np.random.randn(n_steps)
        dW = np.sqrt(dt) * np.random.randn(n_steps)
        
        B_path = np.concatenate([[0], np.cumsum(dB)])
        W_path = np.concatenate([[0], np.cumsum(dW)])
        BH_path = B_path.copy()
        
        S, _ = simulate_rfsv_path(BH_path, B_path, W_path, S0=S0, V0=V0, nu=nu, rho=rho)
        
        log_rets = np.diff(np.log(S))
        
        spot_vars = compute_realized_variances(log_rets, block_size)
        log_vars = np.log(spot_vars)
        log_vars_centered = log_vars - np.mean(log_vars)
        
        H_est, _ = mle_estimator(log_vars_centered)
        estimates.append(H_est)
        estimates_array = np.array(estimates)
    
    return estimates_array

def compute_realized_variances(log_returns, block_size):
    n = len(log_returns)
    dt = T / n
    
    num_blocks = n // block_size
    spot_vars = np.zeros(num_blocks)
    
    for j in range(num_blocks):
        start_idx = j * block_size
        end_idx = start_idx + block_size
        block = log_returns[start_idx:end_idx]
        spot_vars[j] = np.sum(block**2) / (block_size * dt)
        
    return spot_vars

def estimate_params(BH, B, W, m=m, S0=S0, V0=V0_1, nu=nu, rho=rho):
    rho_bar = np.sqrt(1 - rho**2)
    S, V = simulate_rfsv_path(BH, B, W, S0, V0, nu, rho)
    
    log_returns = np.diff(np.log(S))
    dt = 1.0 / (len(B) - 1)
    
    results = {}
    for block_size in m:
        spot_vars = compute_realized_variances(log_returns, block_size)
        
        log_vars = np.log(spot_vars)
        log_vars_centered = log_vars - np.mean(log_vars)
        
        H_est, nu_est = mle_estimator(log_vars_centered)
        results[block_size] = (H_est, nu_est)
    
    return results

######################################################################  Smile  #

def simulate_rough_bergomi_paths(H=H, N=smile_N, V0=V0_2, nu=nu, rho=rho, T=T):
    dt = T / N
    sqrt_dt = np.sqrt(dt)
    t_grid = np.linspace(0, T, N+1)
    correction = 0.5 * nu**2 * (t_grid**(2*H))

    lags = dt * np.arange(1, N+1)
    c_H = np.sqrt(2*H)
    kernel_full = c_H * (lags ** (H - 0.5))

    L = np.zeros((N+1, N))
    for i in range(1, N+1):
        L[i, :i] = kernel_full[:i][::-1] 
        
    return L, correction, dt, sqrt_dt

def compute_call_prices(L, correction, dt, sqrt_dt, K, M=smile_M, N=smile_N, S0=S0, V0=V0_2,  nu=nu, rho=rho):
    rho_bar = np.sqrt(1 - rho**2)
    call_price_sum = np.zeros(len(K))
    
    for m in range(M):
        dW = np.random.normal(0, sqrt_dt, N)
        dB = np.random.normal(0, sqrt_dt, N)
        
        BH_orig = L @ dB
        BH_ant = L @ (-dB)
        
        V_orig = V0 * np.exp(nu * BH_orig - correction)
        V_ant = V0 * np.exp(nu * BH_ant - correction)
        
        logS = np.log(S0) * np.ones(4)  
        
        for i in range(N):
            sqrtV_orig = np.sqrt(V_orig[i])
            sqrtV_ant = np.sqrt(V_ant[i])

            dZ1 = rho * dB[i] + rho_bar * dW[i]     
            dZ2 = rho * (-dB[i]) + rho_bar * dW[i]    
            dZ3 = rho * dB[i] + rho_bar * (-dW[i])   
            dZ4 = rho * (-dB[i]) + rho_bar * (-dW[i]) 
            
            logS[0] += -0.5 * V_orig[i] * dt + sqrtV_orig * dZ1
            logS[1] += -0.5 * V_ant[i] * dt + sqrtV_ant * dZ2
            logS[2] += -0.5 * V_orig[i] * dt + sqrtV_orig * dZ3
            logS[3] += -0.5 * V_ant[i] * dt + sqrtV_ant * dZ4
        
        S_T = np.exp(logS)
        
        payoffs = np.maximum(S_T - K[:, np.newaxis], 0)
        avg_payoffs = np.mean(payoffs, axis=1)
        call_price_sum += avg_payoffs
        
    avg_call = call_price_sum / M
    
    return avg_call

def bs_price_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    call = S * stats.norm.cdf(d1) - K * np.exp(-r*T) * stats.norm.cdf(d2)
    
    return call
    
def implied_vol_call(price, S, K, T, r, lower=1e-6, upper=10):
    f = lambda sig: bs_price_call(S, K, T, r, sig) - price
    
    return opt.brentq(f, lower, upper)

def compute_implied_vols(call_prices, K, S0=S0, T=T, r=0):
    implied_vols = []
    
    for i, k in enumerate(K):
        mc_price = call_prices[i]
        iv = implied_vol_call(mc_price, S0, k, T, r)
        implied_vols.append(iv)
        
    implied_vol_array = np.array(implied_vols)
    
    return implied_vol_array

######################################################################  Monte Carlo Estimators  #

def monte_carlo_bias(H, T, M, n, monte_window, models=models, estimators=estimators):

    estimates = {model: {est: [] for est in estimators} for model in models}

    for _ in range(M):
        _, clean_fbm, _ = generate_fbm(H, n, T)
        contaminated_fbm  = generate_c_fbm(H, n, T)
        BH_rfsv = generate_rfsv(H, n, T)
        fou = generate_fou(H, n, T)

        paths = [clean_fbm, contaminated_fbm, BH_rfsv, fou]

        for model, path in zip(models, paths):

            H_mle = chunk_array_mle(path, monte_window)
            estimates[model]['MLE'].append(H_mle)

            H_pv = chunk_array_power(path, T, monte_window)
            estimates[model]['Power Variation'].append(H_pv)

            H_wave = chunk_array_wavelet(path, monte_window)
            estimates[model]['Wavelet'].append(H_wave)

            H_loglog = chunk_array_loglog(path, monte_window)
            estimates[model]['Log-Log Regression'].append(H_loglog)

    results = {model: {} for model in models}

    for model in models:
        for est in estimators:
            arr = np.array(estimates[model][est])
            arr = arr[~np.isnan(arr)]

            mean_hat, ci_low, ci_high, bias_val, std_val  = confidence_interval(H, len(arr), arr, p=95)
            _, jb_pvalue = stats.jarque_bera(arr)
         

            results[model][est] = {
                'mean_hat': mean_hat,
                'bias': bias_val,
                'std': std_val,
                'conf_interval': (ci_low, ci_high),
                'jb_pvalue': jb_pvalue
            }

    return results, estimates

def collect_monte_carlo_data(H_vals=H_vals, T_vals=T_vals, monte_N=monte_N, monte_M=monte_M, monte_window=monte_window, models=models, estimators=estimators):
    all_results = {}
    all_raw = {}
    
    for T in T_vals:
        print(f"\n\n=== T = {T} years ===")
        n = int(monte_N * T)

        for H in H_vals:
            print(f"\nH = {H:.1f}")

            results, raw_estimates = monte_carlo_bias(H, T, monte_M, n, monte_window)
            all_results[(H, T)] = results
            all_raw[(H, T)] = raw_estimates

    return all_results, all_raw

######################################################################  Estimators  #

def prof_neg_loglik(H, X, t):
    n = len(X)
    S0 = compute_covariance_matrix(H, t)
    
    X_scaled = X * scaling_factor
    S0_scaled = S0 * (scaling_factor ** 2)
    
    L = la.cholesky(S0_scaled, lower=True)
    y = la.solve_triangular(L, X_scaled, lower=True)
    
    quad = np.dot(y, y)
    logdet = 2 * np.sum(np.log(np.diag(L)))
    nu = (n / 2) * np.log(quad / n) + 0.5 * logdet
    
    return nu

def mle_estimator(X, year_correction = 1):
    n = len(X)
    t = np.arange(1, n + 1) / (n/year_correction)
    res = opt.minimize_scalar(
        prof_neg_loglik,
        args=(X, t),
        bounds=(0.0001, 0.9999),
        method='bounded'
    )
    H_hat = res.x

    S0_hat = compute_covariance_matrix(H_hat, t) * (scaling_factor ** 2)
    Lh = la.cholesky(S0_hat, lower=True)
    y_h = la.solve_triangular(Lh, X * scaling_factor, lower=True)
    nu_hat = np.sqrt(np.dot(y_h, y_h) / n)
    
    return H_hat, nu_hat

def log_log_estimator(X, q=2):
    n = len(X)
    k_values = [2**j for j in range(int(np.log2(n)))]
    
    m_q = np.array([np.mean(np.abs(X[k:] - X[:-k])**q) for k in k_values])
    log_m = np.log(m_q)
    log_deltas = np.log(np.array(k_values))
    slope = stats.linregress(log_deltas, log_m)[0]
    H_est = slope / q
    
    return H_est

def power_variation_estimator(X, T, q=2):
    inc = np.diff(np.asarray(X))
    n = len(inc)
    dt = T / n  

    K_q = 2**(q/2) * gamma((q+1)/2) / np.sqrt(np.pi)
    V_q = np.mean(np.abs(inc)**q)

    H = (np.log(V_q) - np.log(K_q)) / (q * np.log(dt))
    return float(np.clip(H, 1e-4, 0.9999))


def wavelet_estimator(X, wavelet='db3'):
    X = np.asarray(X, dtype=float)
    
    W = pywt.Wavelet(wavelet)
    max_level = pywt.dwt_max_level(len(X), W.dec_len)
    
    coeffs = pywt.wavedec(X, W, level=max_level)
    details = coeffs[1:]  

    n_j = np.array([len(d) for d in details], dtype=float)
    S2_j = np.array([np.mean(d**2) for d in details], dtype=float)

    j = -np.log2(n_j)
    y = np.log2(S2_j)
    w_j = np.sqrt(n_j)

    beta_hat, alpha_hat = np.polyfit(j, y, deg=1, w=w_j)
    H_hat = 0.5 * (beta_hat - 1.0)
    
    return float(np.clip(H_hat, 1e-4, 0.9999))

    
    
######################################################################  Monte Carlo Blocking Functions  #

def blocks_with_tail(a, l):
    a = np.asarray(a, dtype=float)
    if a.size < l:
        return np.empty((0, l))  
    k = a.size // l
    starts = list(range(0, max(k-1, 0) * l, l))
    starts.append(a.size - l) 
    return np.stack([a[s:s+l] for s in starts], axis=0)

def chunk_array_wavelet(X, block_size):
    blocks = blocks_with_tail(X, block_size)
    if blocks.size == 0:
        return np.nan

    H_values = []
    for i in range(blocks.shape[0]):
        b = blocks[i]
        H_block = wavelet_estimator(b)
        H_values.append(H_block)

    H_values = np.array(H_values)
    return np.mean(H_values)

def chunk_array_mle(X, block_size):
    blocks = blocks_with_tail(X, block_size)
    if blocks.size == 0:
        return np.nan

    H_values = []
    for i in range(blocks.shape[0]):
        b = blocks[i]
        H_block, _ = mle_estimator(b)
        H_values.append(H_block)

    H_values = np.array(H_values)
    return np.mean(H_values)

def chunk_array_power(X, T, block_size):
    blocks = blocks_with_tail(X, block_size)
    if blocks.size == 0:
        return np.nan

    dt_global = T / (len(X) - 1)
    T_block = dt_global * (block_size - 1)

    H_values = []
    for i in range(blocks.shape[0]):
        b = blocks[i]
        H_block = power_variation_estimator(b, T_block)
        H_values.append(H_block)

    H_values = np.array(H_values)
    return np.mean(H_values)

def chunk_array_loglog(X, block_size):
    blocks = blocks_with_tail(X, block_size)
    if blocks.size == 0:
        return np.nan

    H_values = []
    for i in range(blocks.shape[0]):
        b = blocks[i]
        H_block = log_log_estimator(b)
        H_values.append(H_block)

    H_values = np.array(H_values)
    return np.mean(H_values)
    
######################################################################  Plotting Functions  #

def plot_fbm_paths(H_values, t_full, T):
    n = len(t_full) - 1
    plt.figure(figsize=(10, 6))
    
    for H in H_values:
        _,_,fBm_full = generate_fbm(H, n, T)
        plt.plot(t_full, fBm_full, label=f'H={H}')
    
    plt.title("Fractional Brownian Motion Paths (Cholesky Method)")
    plt.xlabel("Time (t)")
    plt.ylabel("$B_t^H$")
    plt.legend()
    
    plt.grid(True)
    plt.show()

def plot_RFSV_path(BH, B, W, S0=S0, V0=V0_1, nu=nu, rho=rho):
    rho_bar = np.sqrt(1 - rho**2)
    n = len(BH) - 1
    dt = 1.0 / n
    tgrid = np.linspace(0, 1, n + 1)

    V = V0 * np.exp(nu * BH)
    dB = np.diff(B)
    dW = np.diff(W)
    S = np.empty(n + 1)
    S[0] = S0
    for i in range(n):
        S[i+1] = S[i] + S[i] * np.sqrt(V[i]) * (rho * dB[i] + rho_bar * dW[i])

    plt.figure(figsize=(8,6))
    plt.subplot(2,1,1)
    plt.plot(tgrid, V)
    plt.title('Variance Process Simulation with RFSV Model', fontsize = 10)
    plt.xlabel('Time (t)')
    plt.ylabel('$V_t$')
    plt.grid(True)

    plt.subplot(2,1,2)
    plt.plot(tgrid, S)
    plt.title('Stock Price Simulation with RFSV Model', fontsize = 10)
    plt.xlabel('Time (t)')
    plt.ylabel('$S_t$')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

def plot_smile(x, implied_vols, H=H, nu=nu, V0=V0_2, rho=rho):
    plt.figure(figsize=(10, 6))
    plt.plot(x, implied_vols, 'bo-', linewidth=2, markersize=6)
    plt.title(f'Rough Bergomi Implied Volatility Smile (H={H}, ρ={rho})', fontsize=14)
    plt.xlabel('Log-Moneyness (x = log(K/S₀))', fontsize=12)
    plt.ylabel('Implied Volatility', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.axvline(x=0, color='r', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

def plot_spx_hurst(results):
    plt.figure(figsize=(12, 6))

    segments = [r['Segment'] for r in results]
    x = np.arange(len(segments))
    width = 0.8 / len(estimators)

    for i, e in enumerate(estimators):
        vals = [r[e] for r in results]
        plt.bar(x + width * (i - (len(estimators) - 1) / 2), vals, width, label=e, color=colors[i])

    plt.xticks(x, segments)
    plt.ylabel('H Estimate')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.ylim(0, 0.75)
    plt.tight_layout()
    plt.show()

def plot_line_H(results_dict, H_vals=H_vals, colors=colors, estimators=estimators):
    models = list(results_dict[(H_vals[0], T_vals[0])].keys())
    markers = ['o', 's', '^', 'D']
    
    for model in models:
        print(f"Model: {model}")
        plt.figure(figsize=(10, 5))
        plt.xlabel('True Hurst Exponent (H)', fontsize=12)
        plt.ylabel('Estimated H', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        
        plt.plot(H_vals, H_vals, 'k-', linewidth=3, alpha=0.7, label='True H')
        
        plt.ylim(0.0, 1.0)
        plt.yticks(np.arange(0.1, 1.0, 0.2))
        plt.xticks(H_vals)
        
        for est_idx, (estimator, color, marker) in enumerate(zip(estimators, colors, markers)):
            mean_hats = []
            ci_lows = []
            ci_highs = []
            
            for H in H_vals:
                res = results_dict[(H, T_vals[0])][model][estimator]
                mean_hats.append(res['mean_hat'])
                ci_low, ci_high = res['conf_interval']
                ci_lows.append(ci_low)
                ci_highs.append(ci_high)
            
            plt.plot(H_vals, mean_hats, '-', color=color, alpha=0.7, label=estimator)
            plt.fill_between(H_vals, ci_lows, ci_highs, color=color, alpha=0.2)
            plt.scatter(H_vals, mean_hats, marker=marker, s=80, color=color, edgecolor='black', zorder=10)
        
        plt.legend(loc='best')
        plt.tight_layout()
        plt.show()

def plot_boxplots(all_raw, H_vals, T, colors=colors):
    nH = len(H_vals)
    nE = len(estimators)
    group_gap = 1.0
    width = 0.6

    offsets = np.linspace(-1.2, 1.2, nE) * (width / 1.0)

    for model in models:
        print(f"Model: {model}")
        fig, ax = plt.subplots(1, 1, figsize=(10,5))

        data_list = []
        positions = []
        for h_idx, H in enumerate(H_vals):
            group_center = h_idx * (nE + group_gap)
            for e_idx, est in enumerate(estimators):
                pos = group_center + offsets[e_idx]
                positions.append(pos)

                raw = all_raw.get((H, T), {}).get(model, {}).get(est, [])
                arr = np.array(raw, dtype=float)
                arr = arr[~np.isnan(arr)]
                if len(arr) == 0:
                    data_list.append(np.array([np.nan]))
                else:
                    data_list.append(arr)

        bp = ax.boxplot(
            data_list,
            positions=positions,
            widths=width,
            notch=True,
            patch_artist=True,
            showmeans=False,
            meanprops=dict(marker='D', markeredgecolor='k', markerfacecolor='white', markersize=4),
            medianprops=dict(color='black', linewidth=1.5),
            flierprops=dict(marker='.', markersize=2.5, alpha=0.6),
            manage_ticks=False
        )

        for k, patch in enumerate(bp['boxes']):
            color = colors[k % nE]
            patch.set_facecolor(color)
            patch.set_alpha(0.65)

        if 'fliers' in bp:
            for fl in bp['fliers']:
                try:
                    fl.set_marker('.')
                    fl.set_markersize(2.5)
                    fl.set_alpha(0.6)
                except Exception:
                    pass

        group_centers = [h_idx * (nE + group_gap) for h_idx in range(nH)]
        ax.set_xticks(group_centers)
        ax.set_xticklabels([f"{H:.1f}" for H in H_vals])
        ax.set_xlim(- (nE + group_gap), group_centers[-1] + (nE + group_gap))

        ax.set_yticks(H_vals)
        ax.set_ylim(min(H_vals) - 0.05, max(H_vals) + 0.05)

        ax.set_xlabel("True H")
        ax.set_ylabel("Estimated H")
        ax.grid(axis='y', linestyle='--', alpha=0.6)

        proxies = [Patch(facecolor=colors[k], edgecolor='k', alpha=0.6) for k in range(nE)]
        ax.legend(proxies, estimators, loc='upper left', fontsize='small')

        plt.tight_layout()
        plt.show()
        
def plot_histograms(all_raw, H_vals=hist_Vals, T_val=T_vals[0], colors=colors, estimators=estimators, models=models, bins=20):
    target_H = H_vals[:2]

    for model in models:
        print(f"Model: {model}")
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        for ax, H in zip(axes, target_H):
            model_data = all_raw.get((H, T_val), {}).get(model, {})

            all_chunks = []
            for est in estimators:
                arr = np.asarray(model_data.get(est, []), dtype=float)
                arr = arr[~np.isnan(arr)]
                if arr.size > 0:
                    all_chunks.append(arr)

            if not all_chunks:
                ax.set_visible(False)
                continue

            all_vals = np.concatenate(all_chunks)
            xmin, xmax = np.min(all_vals), np.max(all_vals)
            if xmin == xmax:
                xmin -= 0.05
                xmax += 0.05

            pad = 0.05 * (xmax - xmin)
            x_grid = np.linspace(xmin - pad, xmax + pad, 1000)

            local_ymax = 0.0
            for est, color in zip(estimators, colors):
                clean = np.asarray(model_data.get(est, []), dtype=float)
                clean = clean[~np.isnan(clean)]
                if clean.size == 0:
                    continue

                n, _, _ = ax.hist(clean, bins=bins, density=True, alpha=0.35,
                                  color=color, edgecolor="white", linewidth=0.8)
                local_ymax = max(local_ymax, float(np.max(n)))

                mu, sd = np.mean(clean), np.std(clean, ddof=1)
                if sd > 0:
                    y = stats.norm.pdf(x_grid, mu, sd)
                    ax.plot(x_grid, y, color=color, linewidth=2.8, alpha=0.95)
                    local_ymax = max(local_ymax, float(np.max(y)))

            ax.axvline(H, color="black", linestyle="--", linewidth=1.5, label=f"True $H$ = {H:g}")
            ax.legend(loc="upper left", frameon=False, fontsize=10, handlelength=2.5)

            ax.set_xlim(xmin - pad, xmax + pad)
            ax.set_ylim(0, min(local_ymax * 1.10, 150))

            ax.set_xlabel("Estimated H", fontsize=12)
            ax.set_ylabel("Density", fontsize=12)
            ax.grid(True, alpha=0.25)

        plt.tight_layout()
        plt.show()
    
def plot_rmse_convergence(results_rmse, conv_H, conv_T, p=p, models=models, estimators=estimators, colors=colors):
    z = stats.norm.ppf(1 - (100 - p) / 200.0)
    fig, axs = plt.subplots(2, 2, figsize=(16, 12))
    model_positions = {m: divmod(i, 2) for i, m in enumerate(models)}

    for model, (i, j) in model_positions.items():
        ax = axs[i, j]
        for est, color in zip(estimators, colors):
            n_vals = np.array(sorted(results_rmse[model][est].keys()))
            rmse_vals = np.array([results_rmse[model][est][n] for n in n_vals])

            ln_n = np.log(n_vals)
            ln_rmse = np.log(rmse_vals)
            X = sm.add_constant(ln_n)
            fit = sm.OLS(ln_rmse, X).fit()
            slope, se_slope = fit.params[1], fit.bse[1]

            ax.loglog(n_vals, rmse_vals, 'o-', color=color)

            ci_upper = np.exp(fit.params[0] + z*fit.bse[0]) * n_vals**slope
            ci_lower = np.exp(fit.params[0] - z*fit.bse[0]) * n_vals**slope
            ax.fill_between(n_vals, ci_lower, ci_upper, color=color, alpha=0.2)

        ax.set_title(f"{model}")
        ax.set_xlabel("Sample Size (n)")
        ax.set_ylabel("RMSE")
        ax.grid(True, which="both", ls="--")

    plt.show()

def plot_bias_variance_decomposition(results_bias2, results_var, conv_H, conv_T, models=models, estimators=estimators, colors=colors):
    
    fig, axs = plt.subplots(2, 2, figsize=(16, 12))
    model_positions = {m: divmod(i, 2) for i, m in enumerate(models)}
    color_map = dict(zip(estimators, colors))

    for model, (i, j) in model_positions.items():
        ax = axs[i, j]

        for est in estimators:
            c = color_map[est]
            n_vals = np.array(sorted(results_bias2[model][est].keys()))
            vals   = np.array([results_bias2[model][est][n] for n in n_vals])
            ax.loglog(n_vals, vals, linestyle='-', color=c, linewidth=2)

        for est in estimators:
            c = color_map[est]
            n_vals = np.array(sorted(results_var[model][est].keys()))
            vals   = np.array([results_var[model][est][n] for n in n_vals])
            ax.loglog(n_vals, vals, linestyle=':', color=c, linewidth=2)

        ax.set_title(f"{model}")
        ax.set_xlabel("Sample Size (n)")
        ax.set_ylabel("Bias² / Variance")
        ax.grid(True, which="both", ls="--", alpha=0.6)

    style_handles = [
        Line2D([0], [0], linestyle='-', color='black', lw=3, label='Bias²'),
        Line2D([0], [0], linestyle=':', color='black', lw=3, label='Variance'),
    ]
    est_handles = [
        Line2D([0], [0], linestyle='-', color=color_map[e], lw=3, label=e)
        for e in estimators
    ]
    handles = style_handles + est_handles
    labels  = [h.get_label() for h in handles]

    fig.legend(handles, labels,loc='lower center',ncol=len(handles),frameon=False,bbox_to_anchor=(0.5, 0.02),handlelength=2.8,columnspacing=1.0)
    plt.show()

######################################################################  tables  #

def spx_results(data, window_size, trading_days_per_year=252):
    n = len(data)
    seg_len = n // 3
    segments = {
        'First Year': data[:seg_len],
        'Second Year': data[seg_len:2*seg_len],
        'Third Year': data[2*seg_len:],
        'Full Sample': data
    }

    results = []
    for name, seg in segments.items():
        centered = np.log(seg) - np.mean(np.log(seg))
        years = len(seg) / trading_days_per_year

        results.append({
            'Segment': name,
            'Days': len(seg),
            'Years': years,
            'MLE': chunk_array_mle(centered, window_size),
            'Power Variation': chunk_array_power(centered, years, window_size),
            'Wavelet': chunk_array_wavelet(centered, window_size),
            'Log-Log Regression': chunk_array_loglog(centered, window_size)
        })

    col1, colw = 20, 20
    header = "Segment".ljust(col1)
    for e in estimators:
        header += f"{e}".center(colw)
    print(header)
    print("-" * len(header))
    for r in results:
        row = str(r["Segment"]).ljust(col1)
        for e in estimators:
            row += f"{r[e]:.4f}".center(colw)
        print(row)

    return results

def print_all_MC_tables(all_results, H_vals=H_vals, T_vals=T_vals, models=models, estimators=estimators):
    for T in T_vals:
        print(f"\n\n=== T = {T} years ===")
        for H in H_vals:
            print(f"\nH = {H:.1f}")
            results = all_results[(H, T)]

            header = "Estimator".ljust(20)
            for model in models:
                header += f"{model}".center(30)
            print(header)
            print("-" * len(header))

            for est in estimators:
                row1 = est.ljust(20)
                row2 = "bias".ljust(20)
                row3 = "std".ljust(20)
                row4 = "CI".ljust(20)

                for model in models:
                    res = results[model][est]
                    jb_star = format_jb_star(res['jb_pvalue'])

                    row1 += f"{res['mean_hat']:.4f}{jb_star}".center(30)
                    row2 += f"{res['bias']:.4f}".center(30)
                    row3 += f"{res['std']:.4f}".center(30)

                    ci_low, ci_high = res['conf_interval']
                    row4 += f"[{ci_low:.4f}, {ci_high:.4f}]".center(30)

                print(row1)
                print(row2)
                print(row3)
                print(row4)
                if est != estimators[-1]:
                    print("-" * len(header))
            print("-" * len(header))

def create_convergence_table(results_rmse, results_jb, alpha_results, n_max, models=models, estimators=estimators):
    print(f"\n\n=== Convergence Results Table (n = {n_max}) ===")
    
    header = "Estimator".ljust(20)
    for model in models:
        header += f"{model}".center(30)
    print(header)
    print("-" * len(header))

    for est in estimators:
        row1 = est.ljust(20)
        row2 = "rate (±se)".ljust(20) 

        for model in models:
            rmse_val  = results_rmse[model][est][n_max]
            jb_pvalue = results_jb[model][est][n_max]
            jb_star   = format_jb_star(jb_pvalue)
            rmse_str  = f"{rmse_val:.4f}{jb_star}"

            alpha_val, alpha_se = alpha_results[model][est]
            alpha_str = f"{alpha_val:.2f}±{alpha_se:.2f}"

            row1 += rmse_str.center(30)
            row2 += alpha_str.center(30)

        print(row1)
        print(row2)
        if est != estimators[-1]:
            print("-" * len(header))

    print("-" * len(header))
    return alpha_results

######################################################################  Formating  #

def format_jb_star(p):

    if np.isnan(p):
        return ""
    if p < 0.001:
        return "***"
    if p < 0.01:
        return "**"
    if p < 0.05:
        return "*"
    return ""
    
######################################################################  Outputs  ######################################################################


######################################################################  Simulating fBM, Estimating H, Nu w/MLE  #
np.random.seed(seed)

plot_fbm_paths(fbm_H_values, np.linspace(0, T, fbm_n+1), T)
H_hat, nu_hat = mle_estimator(X)
    
print(f"MLE H = {H_hat:.10f}")
print(f"MLE ν = {nu_hat:.10f}")

######################################################################  Simulating RSFV, Estimating H, Nu w/MLE  #

res05 = estimate_params(BH05, B05, W05)
res10 = estimate_params(BH10, B10, W10)
estimates = simulate_paths()

mean, ci_low, ci_high, bias, std = confidence_interval(H_stdBM, n_sim_stdBM, estimates, p)

print("\nH=0.05 results:")
for m_val, vals in res05.items():
        print(f"m={m_val}: H_hat={vals[0]:.4f}, nu_hat={vals[1]:.4f}, relErr={np.abs(vals[0]-0.05)/0.05:.4f}")  
    
print("\nH=0.10 results:")
for m_val, vals in res10.items():
        print(f"m={m_val}: H_hat={vals[0]:.4f}, nu_hat={vals[1]:.4f}, relErr={np.abs(vals[0]-0.1)/0.1:.4f}")

print("\nH=", H_stdBM, "(std BM) results:")
print(f"Mean H_hat: {mean:.4f}")
print(f"Std Dev H_hat: {std:.4f}")
print(f"{p}% CI: [{ci_low:.4f}, {ci_high:.4f}]")
print(f"Bias: {bias:.4f}")

print("\nBH05 Simulation")
plot_RFSV_path(BH05, B05, W05)
    
print("\nBH10 Simulation")
plot_RFSV_path(BH10, B10, W10)

######################################################################  RFSV Implied Volatility Skew  #
    
x = np.linspace(-0.5, 0.5, 11)  
K = S0 * np.exp(x)  
L, correction, dt, sqrt_dt = simulate_rough_bergomi_paths()
call_prices = compute_call_prices(L, correction, dt, sqrt_dt, K)
implied_vols = compute_implied_vols(call_prices, K)
    
results = pd.DataFrame({
    'Log-Moneyness (x)': x,
    'Strike Price (K)': K,
    'Call Price': call_prices,
    'Implied Volatility': implied_vols
})
print("\nImplied Volatility Smile Results:")
print(results.round(4))

plot_smile(x, implied_vols)

######################################################################  Estimating H on spx w/MLE  #

realized_var1 = realized_var3[-251:]

log_realized_var3 = np.log(realized_var3)
centered_log_var3 = log_realized_var3 - np.mean(log_realized_var3)

log_realized_var1 = np.log(realized_var1)
centered_log_var1 = log_realized_var1 - np.mean(log_realized_var1)

H_MLE3spx, nu_MLE3spx  = mle_estimator(centered_log_var3, 3)
H_MLE1spx, nu_MLE1spx  = mle_estimator(centered_log_var1)

print("\nMLE Estimates w/SPX volatility data")
print("3 Years :  " f"H = {H_MLE3spx:.4f},  nu = {nu_MLE3spx:.4f} ")
print("Final Yr:  "f"H = {H_MLE1spx:.4f},  nu = {nu_MLE1spx:.4f} ")

######################################################################  Estimating H on SPX w/Estimators w/Subwindoes  # 

print("\nSPX Hurst Exponent: Yearly Segments vs Full Sample")
spx_results = spx_results(realized_var3, spx_window)
plot_spx_hurst(spx_results)


######################################################################  MC Estimators Bias  #
np.random.seed(seed)

all_results, all_raw = collect_monte_carlo_data()
print_all_MC_tables(all_results)

plot_line_H(all_results)
plot_boxplots(all_raw, H_vals, T_vals[0])
plot_histograms(all_raw)

######################################################################  Estimator Convergence  #

results_rmse = {model: {est: {} for est in estimators} for model in models}
results_bias2 = {model: {est: {} for est in estimators} for model in models}
results_var   = {model: {est: {} for est in estimators} for model in models}
results_jb    = {model: {est: {} for est in estimators} for model in models}

for n in conv_N:
    print(f"\nRunning n={n} (T={conv_T}, H={conv_H})")
    results, _ = monte_carlo_bias(conv_H, conv_T, conv_M, n, conv_window)
    
    for model in models:
        for est in estimators:
            res = results[model][est]
            b2  = res['bias']**2
            var = res['std']**2
            mse = b2 + var
            rmse = np.sqrt(mse)

            results_rmse[model][est][n] = rmse
            results_bias2[model][est][n] = b2
            results_var[model][est][n]   = var
            results_jb[model][est][n]    = res['jb_pvalue']
            
n_max = max(conv_N)
alpha_results = {model: {est: (0.0, 0.0) for est in estimators} for model in models}

for model in models:
    for est in estimators:
        n_vals   = np.array(sorted(results_rmse[model][est].keys()))
        rmse_vals = np.array([results_rmse[model][est][n] for n in n_vals])

        ln_n    = np.log(n_vals)
        ln_rmse = np.log(rmse_vals)
        X = sm.add_constant(ln_n)
        fit = sm.OLS(ln_rmse, X).fit()

        beta = -fit.params[1]
        beta_se = fit.bse[1]
        alpha_results[model][est] = (beta, beta_se)

create_convergence_table(results_rmse, results_jb, alpha_results, n_max)
plot_rmse_convergence(results_rmse, conv_H, conv_T)
plot_bias_variance_decomposition(results_bias2, results_var, conv_H, conv_T)


#end